<a href="https://colab.research.google.com/github/DhafinRA/Apriori/blob/main/Apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
df = pd.read_excel('/content/SERVIS INJEKSI MANUAL BEBEK MATIC.xlsx')

df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,ServTransCode,ServTransDate,Items,NameOfServ,qty,profit
0,SRV001912080003,2019-12-08,Oli Mesin Matic,SERVICE INJEKSI MANUAL BEBEK/MATIC,1,0
1,SRV001912080003,2019-12-08,Grease Cvt,SERVICE INJEKSI MANUAL BEBEK/MATIC,1,0
2,SRV001912080003,2019-12-08,Handel Variasi,SERVICE INJEKSI MANUAL BEBEK/MATIC,1,0
3,SRV001912080003,2019-12-08,Air Radiator,SERVICE INJEKSI MANUAL BEBEK/MATIC,1,0
4,SRV001912120004,2019-12-12,Oli Gardan,SERVICE INJEKSI MANUAL BEBEK/MATIC,1,0


In [4]:
# Stripping extra spaces in the description
df['Items'] = df['Items'].str.strip()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
frequent_implement = apriori(df,min_support=0.07,use_colnames=True)

frequent_implement.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


ValueError: The allowed values for a DataFrame are True, False, 0, 1. Found value SRV001912080003